In [1]:
from LogProcessor import LogAnalytics
from Mongo_DB import Mongo_DB
import requests
import json
import os
import pandas as pd
import streamlit as st

In [14]:
class LogInterface:
    def __init__(self,):
        self.log_processor = LogAnalytics()
        self.DB = Mongo_DB(address='mongodb://localhost:27017/',
                 db_name='call_analytics_tool',
                 collection_name='log_record1',)

    def insert_to_db(self,file_name):
        data = self.log_processor.driver(files_name=file_name)
        file_id = data["file_id"]
        # print(data)
        if  self.DB.check_if_exists(file_id=file_id):
            print('Already Exists')
            return True, 'Data  already exists'
        else:
            temp_=self.DB.insert(data=data)
            if temp_:
                print('Inserted')
                return True, 'Data Added successfully'
            else:
                print('Error')
                return False,'Something went wrong'
    
    def get_complete_data(self,):
        data = self.DB.find()
        # print(data)
        return data

    def get_particular_data(self,file_id):
        data = self.DB.find({'file_id':file_id})
        return data
    
    # in sary functions me mongo db se data lena he ; us data k none ai responsis wali key me jana he; us ki frequency k lie aik df bnaien us se asain hojae gi; df[TEct col].value_counts ; or dobra dictionary bna k return krni he anwar ko 
    def get_none_responsis_pharase_freq(self):
        data = self.DB.find({},['AI None Separater','file_id'])
        try:
            key = list(data[0]['AI None Separater'].keys())[0]
            df_temp = pd.DataFrame(data[0]['AI None Separater'][key])
            df_list = []
            if len(data)>1:
                for i in range(1,len(data)):
                    key = list(data[i]['AI None Separater'].keys())[0]
                    df_temp1 = pd.DataFrame(data[i]['AI None Separater'][key])
                    df_list.append(df_temp1)
            df_concat = pd.concat(df_list)
            df_temp = pd.concat([df_temp,df_concat])
        
            counter =    df_temp['AI bot got this data'].apply(cleanify).value_counts()
            data_response = {"status":True,"data":counter.to_dict(),"msg":"data got"}
            
        except Exception as e:
            print(e)
            data_response = {"status":False,"data":{},"msg":f"You got the error {e}"}
        # print(data_response)
        return data_response
        
    def get_none_responis_word_freq(self):
        data = self.get_none_responsis_pharase_freq()
        if data['status']:
            data = data['data']
            data = list(data.keys())
            list_of_words = [word for string in data for word in split_string_into_words(string)]
            frequency_dict = {}
            for item in list_of_words:
                frequency_dict[item] = frequency_dict.get(item, 0) + 1
            data_response = {"status":True,"data":frequency_dict,"msg":"data got"}
        else:
            data_response = {"status":False,"data":{},"msg":f"You got the error {e}"}

    def get_none_bot_hanged_up(self):
        data = self.DB.find({},['AI None Separater','file_id'])
        try:
            key = list(data[0]['AI None Separater'].keys())[0]
            df_temp = pd.DataFrame(data[0]['AI None Separater'][key])
            df_list = []
            if len(data)>1:
                for i in range(1,len(data)):
                    key = list(data[i]['AI None Separater'].keys())[0]
                    df_temp1 = pd.DataFrame(data[i]['AI None Separater'][key])
                    df_list.append(df_temp1)
            df_concat = pd.concat(df_list)
            df_temp = pd.concat([df_temp,df_concat])
            # please make changes for bot hangedup
            print(df_temp['Next State'],"datas")
            filtered_rows = df_temp[df_temp['Next State'].apply(lambda states: 'Caller Hanged Up' in states)]
            row_data = list(filtered_rows['AI bot got this data'])
            data_response = {"status":True,"data":row_data,"msg":"data got"}
            # print(data_response)
        except Exception as e:
            print(e)
            data_response = {"status":False,"data":[],"msg":f"You got the error {e}"}
        # print(data_response)
        return data_response


In [15]:
logsinterface = LogInterface()
logsinterface.get_none_bot_hanged_up()

0                                 [Hang Up]
1                                 [Hang Up]
2                                 [Hang Up]
3                                 [Hang Up]
4                                 [Hang Up]
                       ...                 
526               [Not Qualified, Negative]
527               [Not Qualified, Negative]
528               [Not Interested, Hang Up]
529    [Not Interested, Positive, Transfer]
530                    [Positive, Transfer]
Name: Next State, Length: 1806, dtype: object
